## Введение в глубокое обучение
### Занятие 6. Автоэнкодеры и векторные представления слов.

##### Автор: [Радослав Нейчев](https://www.linkedin.com/in/radoslav-neychev/), @neychev
##### Credits: embeddings visualization is based on the notebook by [YSDA NLP course](https://github.com/yandexdataschool/nlp_course)

#### План занятия:
0. Повторение основных моментов при работе с текстами
1. Автоэнкодеры, снижение размерности и построение информативных представлений данных
2. Построение информативных представлений слов. Обзор word2vec
3. Работа с предобученными векторными представлениями слов

### 0. Повторение основных моментов при работе с текстами
Вспомним основные отличия текста от табличных данных:
1. Текст представляет собой последовательность токенов из конечного алфавита, т.е. все элементы последовательности принимают дискретные значения.
2. Текст может быть _переменной длины_

__Токен__ – минимальный и неделимый элемент текстовой последовательности. В зависимости от выбора эксперта, в качестве токенов могут выступать как символы, так и морфемы, слова или даже группы слов. 

### 1. Автоэнкодеры, снижение размерности и построение информативных представлений данных
__Go to slides__

### 2. Построение информативных векторных представлений слов. Обзор word2vec.
Человек воспринимает слова знакомого языка как отдельные смысловые единицы. Затем из них складывается смысл всего предложения или текста. Машина, в отличие от человека, не имеет представления о смысле слов, она не владеет языком. Для машины слова являются лишь последовательностю некоторых символов. Чтобы использовать всю доступную информацию при работе с текстом необходимы информативные представления слов. Уже рассмотренное one-hot кодирование имеет несколько минусов.

##### __One-hot кодирование__. 
Каждому токену в словаре был сопоставлен уникальный индекс. Тогда токену можно поставить в соответствие вектор размера словаря, где единственное ненулевое значение стоит на соответствующей токену позиции. Например, слово "самолет" сопоставлено индексу 0, а размерность словаря 5. Данному токену соответствует вектор `[1, 0, 0, 0, 0]`. 
Слово "обед" сопоставлено индексу 4, поэтому ему соответствует вектор `[0, 0, 0, 0, 1]`.

Получается, что векторные представления всех слов ортогональны друг-другу, и слово "кот" удалено от слова "самолет" так же, как и от слова "кошка". Для владеющего языком человека разница между словами очевидна, как и то, что "кошка" и "кот" гораздо ближе друг к другу по смыслу, чем "кошка" и "самолет". При построении информативного векторного представления также хотелось бы установить смысловую близость слов. Учесть связи между словами и их смысл нам поможет word2vec.

#### __Построение эмбеддингов с помощью word2vec__.

__Go to slides__


С этим может помочь простая мысль (озвученная в различных формах множество раз): __слово в значительной мере определяется контекстом, в котором оно встречается__. На основании чего можно сделать простой вывод: для некоторых слов более характерен один контекст, а для других – другой. Именно на этой идее и построен word2vec (как и многие другие эмбеддинги).

По слову можно научиться предсказывать контекст, в котором оно встречается. Конечно, результат не будет идеально точным. Но если модель делает предсказания лучше, чем случайным образом, значит, она улавливает какую-то связь. И тогда внутреннее представление модели для каждого слова и может использоваться в качестве искомого векторного представления, причем и в других задачах.


Формулировка гипотезы выше (слово значительно связано с контекстом) позволяет использовать в качестве обучающей выборки все множество текстов для выбранного языка. Собрание сочинений классиков, статьи в энциклопедии, новостные заметки – все это становится обучающей выборкой. И векторные представления, полученные на основе данных текстов позволяют улавливать связь между этими словами.

In [ ]:
# ! pip install --upgrade nltk gensim bokeh umap-learn

In [1]:
import itertools
import string

import numpy as np
import umap
from nltk.tokenize import WordPunctTokenizer

from matplotlib import pyplot as plt

from IPython.display import clear_output

In [2]:
# download the data:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt -nc
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

File ‘./quora.txt’ already there; not retrieving.


In [4]:
data = list(open("./quora.txt", encoding="utf-8"))
data[55]

'What are all the pros and cons of having dual citizenship?\n'

Произведем токенизацию и базовую предобработку:

In [5]:
tokenizer = WordPunctTokenizer()
print(tokenizer.tokenize(data[50]))
data_tok = [tokenizer.tokenize(x.lower()) for x in data]

['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


In [6]:
len(data)

537272

Для начала, обучим word2vec на доступном наборе данных. Строить для этого модель вручную не понадобится, она уже доступна в `gensim`.

In [7]:
from gensim.models import Word2Vec
model = Word2Vec(data_tok, 
                 vector_size=32,      # embedding vector size
                 min_count=5,  # consider words that occured at least 5 times
                 window=5).wv  # define context as a 5-word window around the target word

/home/rgnejchev/miniconda3/envs/py3_research/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


Теперь нам доступны векторы для любого слова из словаря:

In [15]:
# now you can get word vectors !
model.get_vector('cat')

array([-1.7319554 ,  0.68094164, -0.6284418 ,  0.34835616, -1.575562  ,
        1.608188  , -2.0822666 , -2.123271  ,  1.3782083 ,  1.8109368 ,
        1.532659  , -2.8563302 , -0.4306432 ,  0.91451925,  0.14288747,
       -2.9515886 , -4.737919  ,  0.50100243, -0.3455379 , -2.818347  ,
        2.080946  ,  1.3226941 , -1.4858701 ,  0.46780854,  1.8869146 ,
        1.179702  , -3.5178552 , -0.7419647 , -2.4742515 ,  1.1929777 ,
       -0.13771173, -0.42921153], dtype=float32)

Так как слова представлены векторами, теперь можно вычислить расстояние (или некоторую меру схожести) между ними. Например, можно оценить, какие слова наиболее близки к заданному.

In [20]:
# or query similar words directly. Go play with it!
model.most_similar('pet')

[('prostitute', 0.773679256439209),
 ('dog', 0.761772096157074),
 ('cake', 0.7488067150115967),
 ('parent', 0.7416629195213318),
 ('pants', 0.7283827662467957),
 ('baby', 0.7278494238853455),
 ('bra', 0.7193841338157654),
 ('suit', 0.7120454907417297),
 ('underwear', 0.7102277278900146),
 ('turtle', 0.7071759104728699)]

#### Использование предобученных векторных представлений
Для получения качественных эмбеддингов стоит использовать большие наборы данных. Также бывает полезным использовать данные из определенной предметной области. Конечно, обучение занимает значительное время, поэтому зачастую используются предобученные эмбеддинги для слов.

Загрузим предобученные эмбеддинги небольшой размерности (25). Они были обученны на данных из Twitter. 

In [21]:
import gensim.downloader as api
model = api.load('glove-twitter-25')

In [28]:
model.most_similar(positive=["butter"])#, negative=["code"])

[('peanut', 0.9794958829879761),
 ('cinnamon', 0.9668929576873779),
 ('cheese', 0.9571343064308167),
 ('milk', 0.9549541473388672),
 ('pudding', 0.9447806477546692),
 ('sugar', 0.9443561434745789),
 ('oatmeal', 0.9422078132629395),
 ('cookie', 0.9376214146614075),
 ('corn', 0.9342106580734253),
 ('fruit', 0.9337379932403564)]

#### Визуализация векторных представлений слов

В данный момент каждое слово представлено вектором размерности 25. Для визуализации слов нам понадобится техника снижения размерности. Для простоты можно воспользоваться методом главных компонент, PCA, который также может рассматриваться как линейный автоэнкодер:

$$\|(X W) \hat{W} - X\|^2_2 \to_{W, \hat{W}} \min,$$
где $W$ и $\hat{W}$ – обучаемые параметры.

In [29]:
model.sort_by_descending_frequency()

In [30]:
words = list(model.key_to_index.keys())[:1000]

print(words[::100])

word_vectors = np.asarray([model[x] for x in words])

['<user>', '_', 'please', 'apa', 'justin', 'text', 'hari', 'playing', 'once', 'sei']


In [31]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
pca = PCA(2)
scaler = StandardScaler()

word_vectors_pca = pca.fit_transform(word_vectors)
word_vectors_pca = scaler.fit_transform(word_vectors_pca)

Для визуализации обратимся к замечательной библиотеке `bokeh`. Графики являются интерактивными.

In [32]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

Loading BokehJS ...

In [33]:
draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

Figure(id='1004', ...)

#### Снижение размерности с помощью UMAP
Метод главных компонент – замечательная техника, но он позволяет улавливать лишь линейные зависимости в данных. Обратимся к технике [UMAP](https://habr.com/ru/company/newprolab/blog/350584/), которая учитывает соседей заданных точек. По ссылке выше можно прочитать развернутое описание данной техники.

In [34]:
embedding = umap.UMAP(n_neighbors=5).fit_transform(word_vectors)

In [35]:
draw_vectors(embedding[:, 0], embedding[:, 1], token=words)

Figure(id='1113', ...)

Как можно видеть, слова образуют группы, различимые невооруженным глазом. График интерактивный, при наведении курсора на точку указывается, какому слову она соответствует.

#### Визуализация фраз

В завершение построим векторные представления не только слов, но и целых фраз. Для простоты воспользуемся подходом, аналогичным мешку слов (BoW): мешком эмбеддингов. Каждую фразу представим в виде усредненного эмбеддинга всех входящих в нее слов.

In [36]:
def get_phrase_embedding(phrase):
    """
    Convert phrase to a vector by aggregating it's word embeddings.
    """
    vector = np.zeros([model.vector_size], dtype='float32')
    phrase_tokenized = tokenizer.tokenize(phrase.lower())
    phrase_vectors = [model[x] for x in phrase_tokenized if model.has_index_for(x)]

    if len(phrase_vectors) != 0:
        vector = np.mean(phrase_vectors, axis=0)
    return vector

In [38]:
data[402687]

'What gift should I give to my girlfriend on her birthday?\n'

In [37]:
get_phrase_embedding(data[402687])

array([-0.18204999,  0.30953574,  0.20861094,  0.07982156, -0.22565515,
       -0.33001748,  1.2495784 ,  0.13134292, -0.33788875,  0.06196944,
       -0.231793  ,  0.09389219, -4.9685497 , -0.23611419, -0.32609668,
       -0.092073  ,  0.4407505 , -0.75413746, -0.5389092 , -0.184752  ,
        0.07867809,  0.20018655, -0.16202375,  0.30375698, -0.41255665],
      dtype=float32)

In [40]:
vector = get_phrase_embedding("Another phrase")

In [41]:
vector

array([ 0.37383002,  0.06360999, -0.5646726 ,  0.519285  ,  0.2510745 ,
       -0.221878  ,  1.4463999 , -0.43755502, -0.1671382 , -0.47166997,
       -0.27363   ,  0.31478047, -3.6957998 ,  0.173948  ,  0.28094   ,
        0.775245  ,  0.7375    ,  0.16127   , -0.857635  ,  0.05650002,
        0.114665  ,  0.041415  ,  0.15964499, -0.50083   , -0.475675  ],
      dtype=float32)

Визуализируем лишь небольшое подмножество фраз:

In [42]:
chosen_phrases = data[::len(data) // 1000]

# compute vectors for chosen phrases and turn them to numpy array
phrase_vectors = np.asarray([get_phrase_embedding(x) for x in chosen_phrases])
phrase_vectors_2d = umap.UMAP(n_neighbors=3).fit_transform(phrase_vectors)

In [43]:
draw_vectors(phrase_vectors_2d[:, 0], phrase_vectors_2d[:, 1],
             phrase=[phrase[:50] for phrase in chosen_phrases],
             radius=20,)

Figure(id='1233', ...)

Дополнительно, вы можете попробовать сделать следующее:
* Воспользоваться t-SNE вместо UMAP (требует гораздо больше времени на больших выборках)
* Визуализировать весь набор данных, а не только его часть
* Воспользоваться другими эмбеддингами из `gensim` "model zoo": `gensim.downloader.info()`
* Рассмотреть принципы работы [FastText](https://github.com/facebookresearch/fastText), которые несколько отличаются от word2vec

#### Выводы:
* Использование векторных представлений слов, обученных на значительных объемах текстовых данных, позволяет переносить эти знания в другие задачи.
* При построении некоторого решения для работы с текстами полезно использовать предобученные эмбеддинги. Конечно, их можно и дообучить под задачу.
* Построение эмбеддингов возможно не только для слов, но и для других объектов (текстов, изображений, графов и др.).